<a href="https://colab.research.google.com/github/rtajeong/Hallym_univ_M34/blob/main/lab54_word_vector_rev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 단어벡터 (Word Vector or Word Embedding)
- 빅카인즈 뉴스기사 데이터 활용(https://www.bigkinds.or.kr/)
- 앞에서의 텍스트 코딩 방식인 One-hot encoding, BoW(단어모음), 문서-단어 행렬 방식은 모두 단어마다 고유번호를 배정하여 사용함. 그러나 이 번호들에는 아무런 의미가 포함되어 있지 않고 단지 인덱스 역할만.
- 대신에 단어를 의미있는 숫자들의 집합, 즉, 벡터로 표현하는 방법이 소개됨 - 단어벡터 (Word Vector)

### Word Vector(단어벡터)
- 단어를 고차원 공간상의 벡터로 표현함으로 단어간 거리 표현 가능. 차원이 높을수록 정교한 의미 구분 가능.
- 보통 50~300 개 정도의 차원을 사용함
- 단어간의 거리 (유사도) 뿐 아니라 방향성(벡터)도 찾을 수 있음.
- 단어벡터의 각 성분이 어떤 의미를 갖는지는 알 수 없다.
*****************

# 한국어 뉴스를 이용한 단어 벡터 생성
- 단어 추출: konlpy 의 kkma() 사용
- 단어 벡터 생성: gensim 의 word2vec() 사용

In [14]:
!pip install konlpy

In [15]:
# gensim: library for topic modeling, document indexing and similarity retrieval
!pip install gensim

In [16]:
from konlpy.tag import Kkma     # 형태소 분석 및 자연어 처리 모듈 (서울대)
from gensim.models.word2vec import Word2Vec
kkma = Kkma()

In [17]:
# KoNLPy: 형태소 (뜻을 가진 최소 언어 단위) 분석
# kkma: Korean morpheme analyzer
# https://konlpy.org/ko/latest/morph/#pos-tagging-with-konlpy
print(kkma.sentences('네, 안녕하세요. 반갑습니다.'))
print(kkma.nouns('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))
print(kkma.pos('오류보고는 실행환경, 에러메세지와함께 설명을 최대한상세히!^^'))
# pos (part-of-speech): 품사 (명사, 동사, ...)

['네, 안녕하세요.', '반갑습니다.']
['질문', '건의', '건의사항', '사항', '깃헙', '이슈', '트래커']
[('오류', 'NNG'), ('보고', 'NNG'), ('는', 'JX'), ('실행', 'NNG'), ('환경', 'NNG'), (',', 'SP'), ('에러', 'NNG'), ('메세지', 'NNG'), ('와', 'JKM'), ('함께', 'MAG'), ('설명', 'NNG'), ('을', 'JKO'), ('최대한', 'NNG'), ('상세히', 'MAG'), ('!', 'SF'), ('^^', 'EMO')]


## IT 뉴스기사

In [18]:
!wget https://bit.ly/2X7UON2 -O news2018.xlsx   # wget 도 동일

--2025-09-20 10:16:46--  https://bit.ly/2X7UON2
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.dropbox.com/s/gr5jhlshp8m3iqn/NewsResult_20181128-20190228.xlsx [following]
--2025-09-20 10:16:46--  https://www.dropbox.com/s/gr5jhlshp8m3iqn/NewsResult_20181128-20190228.xlsx
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p2s6opftbedrfcrzyxbgi/NewsResult_20181128-20190228.xlsx?rlkey=hkpxrl46nqa0mppsqtbkuirdf [following]
--2025-09-20 10:16:46--  https://www.dropbox.com/scl/fi/p2s6opftbedrfcrzyxbgi/NewsResult_20181128-20190228.xlsx?rlkey=hkpxrl46nqa0mppsqtbkuirdf
Reusing existing connection to www.dropbox.com:443.
HTTP request se

In [19]:
import pandas as pd
news = pd.read_excel("news2018.xlsx")

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
news.head(2)

In [21]:
news.shape

(1543, 19)

In [22]:
news["본문"][:4]

,본문
0,- 비핵화 수준 상응 조치 놓고\n- 양국 협상팀 막판까지 ‘밀당’\n- 1차 때와...
1,김정은 국무위원장이 27일 시작되는 제2차 북미정상회담 성공을 위해 심혈을 기울이고...
2,북미가 처음으로 정상 간 단독회담과 만찬을 가지며 또다시 새로운 역사 창조에 나섰다...
3,지난해 9월 남북정상회담 당시 리선권 북한 조국평화통일위원장의 '냉면' 발언으로 정...


In [23]:
sentence_list = []
for sent in news["본문"]:

    ## 명사만 추출
    sent_kkma_pos = kkma.nouns(sent)
    word_list = []
    for word_pos in sent_kkma_pos:
        word_list.append(word_pos)

    sentence_list.append(word_list)


In [27]:
import numpy as np

In [26]:
np.shape(sentence_list[0])

(61,)

In [29]:
len(sentence_list)

1543

In [35]:
news.shape

(1543, 19)

In [36]:
[len(w) for w in sentence_list][:10]

[61, 50, 63, 48, 44, 38, 38, 60, 52, 39]

In [37]:
print(sentence_list[0])

['비핵화', '수준', '상응', '조치', '양국', '협상', '협상팀', '팀', '막판', '당', '1', '1차', '차', '때', '시간', '조율', '단계적', '접근', '동의', '예상', '종전', '종전선언', '선언', '연락', '연락사무소', '사무소', '개설', '등', '조건', '조건부', '부', '제재', '완화', '명시', '가능성', '북미', '북미회담', '회담', '빅딜', '성공', '김', '김정은', '정은', '답방', '결과', '연관', '북한', '국무', '국무위원장', '위원장', '도', '도널드', '널드', '트럼프', '미국', '대통령', '27', '27일', '일', '친교', '만찬']


- word2vec(): Its input is a text corpus and its output is a set of vectors.
- Word embedding via word2vec can make natural language computer-readable, then further implementation of mathematical operations on words can be used to detect their similarities. A well-trained set of word vectors will place similar words close to each other in that space. For instance, the words women, men, and human might cluster in one corner, while yellow, red and blue cluster together in another
- Word2Vec의 기본 가정은 유사한 컨텍스트를 공유하는 두 단어도 유사한 의미를 공유하고 결과적으로 모델에서 유사한 벡터 표현을 공유한다는 것이다. 예를 들어 : "dog", "puppy"및 "pup"은 "good", "fluffy"또는 "cute"와 같은 유사한 주변 단어와 함께 유사한 상황에서 자주 사용되며 따라서 Word2Vec에 따르면 유사한 벡터 표현을 공유한다.

이 가정에서 Word2Vec을 사용하여 데이터 세트에서 단어 간의 관계를 알아 내고, 단어 간의 유사성을 계산하거나, 텍스트 분류 또는 클러스터링과 같은 다른 응용 프로그램의 입력으로 해당 단어의 벡터 표현을 사용할 수 있다.

## Exercise

In [38]:

# https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
from gensim.models import Word2Vec
# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
			['this', 'is', 'the', 'second', 'sentence'],
			['yet', 'another', 'sentence'],
			['one', 'more', 'sentence'],
			['and', 'the', 'final', 'sentence']]
# train model
model = Word2Vec(sentences, min_count=1, vector_size=5)

In [39]:
# summarize the loaded model
print(model)

Word2Vec<vocab=14, vector_size=5, alpha=0.025>


In [40]:
# summarize vocabulary (to see learned vocabulary of tokens (words) )
words = list(model.wv.index_to_key)
print(words)

['sentence', 'the', 'is', 'this', 'final', 'and', 'more', 'one', 'another', 'yet', 'second', 'word2vec', 'for', 'first']


In [41]:
# access vector for one word
print(model.wv['sentence'])

[-0.01072454  0.00472863  0.10206699  0.18018547 -0.186059  ]


In [42]:
# save model
model.save('model.bin')

In [43]:
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

Word2Vec<vocab=14, vector_size=5, alpha=0.025>


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

X = model.wv[model.wv.index_to_key]
pca = PCA(n_components=2)
result = pca.fit_transform(X)

plt.scatter(result[:, 0], result[:, 1])
words = list(model.wv.index_to_key)
for i, word in enumerate(words):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]))
plt.show()

## Get back to the problem

In [45]:
model = Word2Vec(sentence_list, vector_size=10)

In [46]:
model.wv['한국']

array([-0.2352448 , -1.4341162 ,  0.13588016,  1.820714  , -1.430692  ,
        0.73804814, -0.06287798,  1.2314048 , -3.5726593 , -1.8600773 ],
      dtype=float32)

In [47]:
model.wv.most_similar("한국")   # cosine similarity

[('자유', 0.9911430478096008),
 ('자유한국', 0.9818752408027649),
 ('자유한국당', 0.980215311050415),
 ('당', 0.9748926758766174),
 ('전당', 0.9687797427177429),
 ('대회', 0.9685946702957153),
 ('이해', 0.9617274403572083),
 ('민주당', 0.9591065049171448),
 ('전당대회', 0.9575591087341309),
 ('경원', 0.9570915699005127)]

In [48]:
print(model.wv.similarity("미국","미국"))
print(model.wv.similarity("미국", "일본"))
print(model.wv.similarity("일본", "한국"))
print(model.wv.similarity("트럼프","김정은"))

1.0
0.61213076
0.724641
0.68024266


## 문장 유사도
- NLTK (Natural Language ToolKit) 라이브러리 사용

In [50]:
# simple exercise
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

sentence = "At eight o'clock on Thursday morning Arthur didn't feel very good."
tokens = nltk.word_tokenize(sentence)
tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


['At',
 'eight',
 "o'clock",
 'on',
 'Thursday',
 'morning',
 'Arthur',
 'did',
 "n't",
 'feel',
 'very',
 'good',
 '.']

In [52]:
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')
tagged = nltk.pos_tag(tokens)
tagged[0:6]

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('At', 'IN'),
 ('eight', 'CD'),
 ("o'clock", 'NN'),
 ('on', 'IN'),
 ('Thursday', 'NNP'),
 ('morning', 'NN')]

In [53]:
# edit_distance: 문자열이 얼마나 다른지 편집거리를 이용해 유사도 판단
import nltk
sentence_list = ["우리 모두 함께 놀자",
                 "모두 같이 놀자",
                 "놀자",
                 "모두 다 같이"]

for i in sentence_list:
    print("'", i, "'")
    for j in sentence_list:
        print("\t", j, " : ", end='')
        print(nltk.edit_distance(i, j), )
    print()

' 우리 모두 함께 놀자 '
	 우리 모두 함께 놀자  : 0
	 모두 같이 놀자  : 5
	 놀자  : 9
	 모두 다 같이  : 7

' 모두 같이 놀자 '
	 우리 모두 함께 놀자  : 5
	 모두 같이 놀자  : 0
	 놀자  : 6
	 모두 다 같이  : 4

' 놀자 '
	 우리 모두 함께 놀자  : 9
	 모두 같이 놀자  : 6
	 놀자  : 0
	 모두 다 같이  : 7

' 모두 다 같이 '
	 우리 모두 함께 놀자  : 7
	 모두 같이 놀자  : 4
	 놀자  : 7
	 모두 다 같이  : 0



- 단어벡터와 코사인 유사도 사용
- ![image.png](attachment:image.png)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

sentence_list = ['우리 모두 함께 놀자',
                 '모두 같이 놀자']

tfidf = TfidfVectorizer()
tfidf_vec = tfidf.fit_transform(sentence_list)

count = CountVectorizer()
count_vec = count.fit_transform(sentence_list)

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

tfidf_cosine = cosine_similarity(tfidf_vec[0].toarray(), tfidf_vec[1].toarray())[0][0]
count_cosine = cosine_similarity(count_vec[0].toarray(), count_vec[1].toarray())[0][0]

print("tfidf consine similarity : ", tfidf_cosine)
print("coiunt consine similarity : ", count_cosine)

tfidf consine similarity :  0.4112070550676187
coiunt consine similarity :  0.5773502691896258


In [56]:
# 직접 구현시 코사인 유사도
import math
def my_cosine_similarity(v1,v2):

    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

my_tfidf = my_cosine_similarity(tfidf_vec.toarray()[0], tfidf_vec.toarray()[1])
my_count = my_cosine_similarity(count_vec.toarray()[0], count_vec.toarray()[1])

print("my tfidf consine similarity : ", my_tfidf)
print("my coiunt consine similarity : ", my_count)

my tfidf consine similarity :  0.4112070550676187
my coiunt consine similarity :  0.5773502691896258


In [57]:
# 자카드 유사도 계산

sentence_list = ['우리 모두 함께 놀자','모두 같이 놀자']
def get_jaccard_sim(str1, str2):
    a = set(str1.split())
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

get_jaccard_sim(sentence_list[0], sentence_list[1])

0.4